In [1]:
#copying imoprts from file
import requests
import json
import os
from fastapi import FastAPI
import pandas as pd

In [25]:
elements = ["fire", "water", "air", "earth"]
element_mappings = {
    "BIOMASS" : "fire",
    "NUCLEAR" : "fire",
    "SOLAR" : "fire",
    "FOSSIL OIL" : "earth",
    "FOSSIL HARD COAL" : "earth",
    "FOSSIL GAS" : "earth",
    "HYDRO PUMPED STORAGE" : "air",
    "HYDRO RUN-OF-RIVER AND POUNDAGE" : "air",
    "WIND ONSHORE" : "water",
    "WIND OFFSHORE" : "water",
}

In [29]:
def response_to_df(response):
    rows = [] 
    for energy in response:
        energy_name = energy["psrType"]
        halfhr_use  = energy["halfHourUsage"]
        halfhr_perc = energy["halfHourPercentage"]
        day_use     = energy["twentyFourHourUsage"]
        day_perc    = energy["twentyFourHourPercentage"]

        element = element_mappings.get(energy_name.upper(), "ELEMENT NOT FOUND")
        
        rows.append({
            "energy" : energy_name,
            "30min_use" : halfhr_use,
            "30min_perc": halfhr_perc,
            "24hr_use" : day_use,
            "24hr_perc" : day_perc,
            "element" : element
        })
        
    response_df = pd.DataFrame(rows)
    return response_df

In [37]:
def get_current_data_as_elements():
    filename = "cached_live_response.json"
    path_to_data = os.path.join("..", "data")
    filepath = os.path.join(path_to_data, filename)
    # with open(filepath, 'w') as file_cache_out:
    #     json.dump(response, file_cache_out)
    with open(filepath, 'r') as file_cache_in:
        response = json.load(file_cache_in)
    
    response_df = response_to_df(response)
    return response_df
response_df = get_current_data_as_elements()
response_df

,energy,30min_use,30min_perc,24hr_use,24hr_perc,element
0,Solar,3424.000,14.8,117438.000,15.1,fire
1,Wind Offshore,19.416,0.1,25118.242,3.2,water
2,Wind Onshore,245.395,1.1,34145.542,4.4,water
3,Other,279.000,1.2,15828.000,2.0,ELEMENT NOT FOUND
4,Nuclear,4533.000,19.6,211559.000,27.2,fire
5,Fossil Oil,0.000,0.0,0.000,0.0,earth
6,Fossil Gas,11881.000,51.4,276886.000,35.6,earth
7,Fossil Hard coal,298.000,1.3,1225.000,0.2,earth
8,Hydro Run-of-river and poundage,409.000,1.8,16735.000,2.2,air
9,Hydro Pumped Storage,427.000,1.8,6253.000,0.8,air


In [60]:
from datetime import datetime
import pytz #for timezones

def get_current_uk_time():
    uk_tz = pytz.timezone('Europe/London')
    uk_time = datetime.now(uk_tz)
    format_time = uk_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    return format_time
get_current_uk_time()

'2024-07-08T11:25:30Z'

In [69]:
total_df = response_df.groupby("element").sum()
total_df

,energy,30min_use,30min_perc,24hr_use,24hr_perc
element,,,,,
ELEMENT NOT FOUND,Other,279.000,1.2,15828.000,2.0
air,Hydro Run-of-river and poundageHydro Pumped St...,836.000,3.6,22988.000,3.0
earth,Fossil OilFossil GasFossil Hard coal,12179.000,52.7,278111.000,35.8
fire,SolarNuclearBiomass,9577.000,41.4,402136.000,51.7
water,Wind OffshoreWind Onshore,264.811,1.2,59263.784,7.6


In [87]:
for source in response_df.query("element == 'fire'").to_dict(orient="records"):
    source_dict = {
        "name" : source["energy"],
        "30_min" : source["30min_use"],
        "24_hours" : source["24hr_use"]
    }

    print(type(row))
    print(source_dict)

<class 'dict'>
{'name': 'Solar', '30_min': 3424.0, '24_hours': 117438.0}
<class 'dict'>
{'name': 'Nuclear', '30_min': 4533.0, '24_hours': 211559.0}
<class 'dict'>
{'name': 'Biomass', '30_min': 1620.0, '24_hours': 73139.0}


In [90]:
four_elements = ["fire","air", "water", "earth"]
def build_output(response_df):
    out_dict = {}
    #add timestamp
    out_dict["timestamp"] = get_current_uk_time()
    #add overall totals
    out_dict["total"] = {
        "30_min" : response_df["30min_use"].sum(),
        "24_hours" : response_df["24hr_use"].sum()
    }
    
    #calculate totals per element 
    total_df = response_df.groupby("element").sum()

    elements_dicts_out = {}
    for element in four_elements:
        #get the per-source data
        sources_list = []
        for source in response_df.query(f"element == '{element}'").to_dict(orient="records"):
            sources_list.append({
                "name" : source["energy"],
                "30_min" : source["30min_use"],
                "24_hours" : source["24hr_use"]
            })
        #build the output for this element
        element_dict = {
            "30_min" : total_df.loc[element, "30min_use"],
            "24_hours" : total_df.loc[element, "24hr_use"],
            "sources" : sources_list
        }
        elements_dicts_out[element] = element_dict

    out_dict["elements"] = elements_dicts_out
    return out_dict
display(build_output(response_df))

{'timestamp': '2024-07-08T11:46:24Z',
 'total': {'30_min': np.float64(23135.811),
  '24_hours': np.float64(778326.784)},
 'elements': {'fire': {'30_min': np.float64(9577.0),
   '24_hours': np.float64(402136.0),
   'sources': [{'name': 'Solar', '30_min': 3424.0, '24_hours': 117438.0},
    {'name': 'Nuclear', '30_min': 4533.0, '24_hours': 211559.0},
    {'name': 'Biomass', '30_min': 1620.0, '24_hours': 73139.0}]},
  'air': {'30_min': np.float64(836.0),
   '24_hours': np.float64(22988.0),
   'sources': [{'name': 'Hydro Run-of-river and poundage',
     '30_min': 409.0,
     '24_hours': 16735.0},
    {'name': 'Hydro Pumped Storage', '30_min': 427.0, '24_hours': 6253.0}]},
  'water': {'30_min': np.float64(264.81100000000004),
   '24_hours': np.float64(59263.784),
   'sources': [{'name': 'Wind Offshore',
     '30_min': 19.416,
     '24_hours': 25118.242},
    {'name': 'Wind Onshore', '30_min': 245.395, '24_hours': 34145.542}]},
  'earth': {'30_min': np.float64(12179.0),
   '24_hours': np.floa

In [ ]:
{
  "timestamp": "2024-07-05T12:00:00Z",
  "total": {
    "30_min": 21773.705,
    "24_hours": 858384.531
  },
  "elements": {
    "fire": {
      "30_min": 14889.0,
      "24_hours": 424384.0,
      "sources": [
        {
          "name": "BIOMASS",
          "30_min": 1292.0,
          "24_hours": 70825.0
        },
        {
          "name": "SOLAR",
          "30_min": 8429.0,
          "24_hours": 118236.0
        },
        {
          "name": "NUCLEAR",
          "30_min": 5168.0,
          "24_hours": 235323.0
        }
      ]
    },
    "wind": {
      "30_min": 4798.705,
      "24_hours": 231091.531,
      "sources": [
        {
          "name": "WIND OFFSHORE",
          "30_min": 284.69,
          "24_hours": 52659.945
        },
        {
          "name": "WIND ONSHORE",
          "30_min": 4514.015,
          "24_hours": 178431.586
        }
      ]
    },
    "water": {
      "30_min": 99.0,
      "24_hours": 7193.0,
      "sources": [
        {
          "name": "HYDRO RUN-OF-RIVER AND POUNDAGE",
          "30_min": 99.0,
          "24_hours": 5965.0
        },
        {
          "name": "HYDRO PUMPED STORAGE",
          "30_min": 0.0,
          "24_hours": 1228.0
        }
      ]
    },
    "earth": {
      "30_min": 2227.0,
      "24_hours": 174156.0,
      "sources": [
        {
          "name": "FOSSIL GAS",
          "30_min": 2227.0,
          "24_hours": 168293.0
        },
        {
          "name": "FOSSIL HARD COAL",
          "30_min": 0.0,
          "24_hours": 5863.0
        },
        {
          "name": "FOSSIL OIL",
          "30_min": 0.0,
          "24_hours": 0.0
        }
      ]
    }
  }
}